In [5]:
import duckdb

In [6]:
con = duckdb.connect(database='dados_duckdb.db',read_only=False)

In [16]:
df = con.execute("""
                 SELECT * FROM (
                 SELECT *,ROW_NUMBER() OVER(PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                 FROM bronze_z0019
                 WHERE data_ingestao >= '2025-09-04'
                 ) WHERE row = 1
                 """).fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-09-04 22:22:07.231726,1
1,10004,SERRA,BT50,100,200,z0019_2.csv,2025-09-04 22:30:13.619129,1
2,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-09-04 22:30:13.619129,1
3,10003,PREGO,BT10,100,60,z0019_2.csv,2025-09-04 22:30:13.619129,1
4,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-09-04 22:22:07.231726,1


In [20]:
df_final = df.drop(columns=['nome_arquivo','data_ingestao','row'])

df_final = df_final.rename(columns={"NATBR": "id"})
df_final = df_final.rename(columns={"MAKTX": "nm_produto"})
df_final = df_final.rename(columns={"WERKS": "id_categoria"})
df_final = df_final.rename(columns={"MAINS": "id_fornecedor"})
df_final = df_final.rename(columns={"LABST": "vl_preco"})

df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500
1,10004,SERRA,BT50,100,200
2,10005,MACHADO,BT50,100,100
3,10003,PREGO,BT10,100,60
4,10001,PARAFUSO,BT10,100,100


In [27]:
df2.dtypes

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor      int64
vl_preco         float64
dtype: object

In [30]:
df2 = df_final

df2 = df2.astype(
    {
        'id': int,
        'nm_produto': str,
        'id_categoria': str,
        'id_fornecedor': int,
        'vl_preco': float
    }

)

df2.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500.0
1,10004,SERRA,BT50,100,200.0
2,10005,MACHADO,BT50,100,100.0
3,10003,PREGO,BT10,100,60.0
4,10001,PARAFUSO,BT10,100,100.0


In [32]:
con.execute("""
 CREATE TABLE IF NOT EXISTS produtos (
            id BIGINT, 
            nm_produto TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            vl_preco FLOAT
            )
""")

In [33]:
df2.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500.0
1,10004,SERRA,BT50,100,200.0
2,10005,MACHADO,BT50,100,100.0
3,10003,PREGO,BT10,100,60.0
4,10001,PARAFUSO,BT10,100,100.0


In [34]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [36]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500.0
1,10004,SERRA,BT50,100,200.0
2,10005,MACHADO,BT50,100,100.0
3,10003,PREGO,BT10,100,60.0
4,10001,PARAFUSO,BT10,100,100.0


In [37]:
con.close()